In [1]:
import torch
import numpy as np
import sys
sys.path.append('..')
from utils.helper_fcts import display_state
from IPython.display import clear_output
import os 
import time
from agent_policy.policy_base_class import Policy, DQN_Agent
from agent_policy.policy_sac import SAC_Agent
from utils.helper_fcts import display_state, class_to_dict, parse_arguments
import random, torch, os, time
from datetime import datetime
import numpy as np
from envs.environment_t import Environment 
from agent_policy.policy_base_class import Policy, DQN_Agent 
from agent_policy.train_policy_cfg import DQNCfg
from utils.helper_fcts import class_to_dict, parse_arguments
from agent_policy.train import *
import sys

Add the name of the model you want here: 

In [2]:
filename = "Ben_6chan_conv_05-06-2023-14-38-01.pt"
# Specify the locaiton of the safe states
current_folder = globals()['_dh'][0]
project_folder = os.path.dirname(current_folder)
model_dir = os.path.join(project_folder,"model", filename)
### models for variant 0
variant = 1  # problem variant
mode = "testing"


# Var 1 
# /media/almo/Windows/Users/Thien/Documents/MSNE/SS23_DRLSem/drl_grid_world/model/30-07-2023-18-04-57_var1/BIG_SAC_Conv1_1_4duel_var_1_30-07-2023-18-04-57_actorbest_.pt


In [4]:
model_dir = input()
print(model_dir)

/media/almo/Windows/Users/Thien/Documents/MSNE/SS23_DRLSem/drl_grid_world/model/30-07-2023-18-04-57_var1/BIG_SAC_Conv1_1_4duel_var_1_30-07-2023-18-04-57_actorbest_.pt


Create your agent there. Sadly, you would need to recreate the agent so that you the saved model will have a shell to paste into. The easiest way i found is finding the hyperparams on wandb

In [5]:

policy_name = 'sac'
neural_net_type = 'duel' # choose between 'conv' and 'fc' and 'duel'
device="cuda"
# pre_trained_model = None # leave to None if we're not doing transfert learning
### problem variant

## END OF PARAMETERS ##

# other parameters 
data_dir = './data'
policy_dict = {"dqn": 'DQN_Agent', "sac": 'SAC_Agent'} 
variable_lr = False # schedule for the learning rate

### seeding ###
seed = 1  # set seed to allow for reproducibility of results
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed=seed)

args = parse_arguments([])
obs_model = args.obs_model

if neural_net_type == 'conv' or neural_net_type == 'duel':
    layers_dims = None
    conv_architecture = args.conv_arch
    n_obs = (1, conv_architecture['num_channel'], 5, 5)
    env = Environment(variant, 
                      data_dir, 
                      neural_net_type=neural_net_type, 
                    #   obs_model=obs_model
                      )
    state = env.reset(mode)
    
policy_class = eval(policy_dict[policy_name])
agent: Policy = policy_class(args, 
                            #   num_inputs=n_obs, 
                            #   num_outputs=args.n_actions,
                            #   device=device,
                            #   layer_dims=layers_dims,
                            #   activation='relu',  
                            #   weight_init=True,
                            # #  pre_trained_model=pre_trained_model,
                            #   conv_architecture=conv_architecture,
                            #   variable_lr=False,
                            #   dropout_rate=args.dropout_rate,
                            #   net_type=neural_net_type,
                              )


saved_model = torch.load(model_dir)
agent.actor.load_state_dict(saved_model)
agent

-1


In [6]:
agent.actor

ActorNetwork(
  (activation): ReLU()
  (feature_ex_block): Sequential(
    (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 32, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=5, bias=True)
  )
  (softmax): Softmax(dim=1)
)

## Below is the newer code visualize the model with the new way of saving

In [7]:
model = agent
add_viz = False
test_rew = 0. 
eps_rew = []
env.test_episode_counter = 0

for idx in range(100):
    obs = env.reset(mode)
    state = obs.to(device)
    test_rew = 0
    for j in range(201):  # loop over 200 steps per episode
        model.actor.eval()
        act = model.select_action(state, greedy=False)  # TODO: get action for the obs from your trained policy
        rew, next_obs, done = env.step(act)  # take one step in the environment

        test_rew += rew
        state = next_obs.to(device)
        # visualization
        clear_output(wait=True)
        
        if add_viz:
            display_state(state=env.get_state(), episode=mode, time =env.step_count,episode_nr=idx, total_rew=test_rew)
            time.sleep(0.0001)  
    eps_rew.append(test_rew)
    if not add_viz:
        print("new episode # ", idx, f' with reward: {eps_rew[idx]} ',  )

avg_test_rew = np.sum(eps_rew) / 100  # compute the average reward per episode

print(avg_test_rew) 

new episode #  99  with reward: 354.5 
312.61


In [17]:
avg_test_rew
## Var0: 182
## Var1: 316 
## Var2: 195




195.095